# Action Menu - Alice

## Role - Requester

In this notebook we'll be going through the Action Menu Protocol. For details on the protocol, please refer to the [RFC](https://github.com/hyperledger/aries-rfcs/tree/master/features/0509-action-menu).

A requirement for this protocol to work is an active DIDComm communication channel between Alice and Bob. To achieve that, you should have completed the [Establishing a Connection](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial.

## 1. Initialise Alice controller

We begin by initializing the controller for the Alice agent.

In [ ]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)
    

## 2. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

In [ ]:
def actionmenu_handler(payload):
    print(f"Action Menu Webhook: {payload}\n")
    menu = payload["menu"]
    print(f"Menu: {menu}\n")
    menu_options = payload["menu"]["options"]
    print(f"Options: {menu_options}")

actionmenu_listener = {
    "topic": "actionmenu",
    "handler": actionmenu_handler
}

agent_controller.register_listeners([actionmenu_listener], defaults=True)

## 3. Check the agent has an active connection

An active connection between Alice and Bob is required. You can either:

- complete the [Establishing a Connection](http://127.0.0.1:8888/notebooks/1%20Basic%20%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial, or
- running the python script `create_connection.py` in the setup folder

In [ ]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")

## 4. Request Action Menu from Bob

Alice begins by making a request for the action menu from Bob.

In [ ]:
await agent_controller.action_menu.request_menu(connection_id)

## 5. Continue with step 6 of [Bob's notebook](http://localhost:8889/lab/tree/5%20Advanced%20Concepts/Part%201%20-%20Action%20Menu.ipynb)

## 8. Perform action upon receiving action menu

Upon receiving the action menu from Bob, Alice can now perform an action.

In [ ]:
# retrieve the menu options sent from Bob
menu = await agent_controller.action_menu.get_menu(connection_id)
menu

In [ ]:
menu_options = menu["result"]["options"]
menu_options

In [ ]:
# as an example, we work with the first menu option
menu_params = {
    "begin_issue_cred": "True"
}

await agent_controller.action_menu.perform(
    connection_id=connection_id,
    menu_params=menu_params,
    menu_option_name=menu_options[0]["name"]
)

## 9. Close the active menu

Once the requester is done with the action menu, the menu can now be closed.

In [ ]:
# close active menu
await agent_controller.action_menu.close_menu(connection_id)

Retrieving the active menu now would return `None`.

In [ ]:
menu = await agent_controller.action_menu.get_menu(connection_id)
menu

## End of Tutorial

Be sure to terminate the controller before running another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)